<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Custom Plugins for Decision Recommendation 

Decision Intelligence applied in this module:  
* Adding current knowledge graph (internet) in order to arrive at conclusions  
* Recommending a decision based on the information extracted from the knowledge graph (internet)
* Providing citations to back up the decision conclusion  

This module will show how to perform a more advanced search using the Bing Search SDK. Any .NET SDK that is used for decision making, business processes, interacts with data etc. can be integrated into AI Orchestration with Semantic Kernel. 

**What is the difference between this module and the previous?** In the previous examples, the Bing Search plugin that comes part of the Semantic Kernel framework was used. It has limited parameters, thus configurability. This custom plugin will be used to recommend whether to make a decision if a baseball player should be inducted into the Hall of Fame.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets
* Use the API configuration to build the Semantic Kernel orchestrator
* The configuration builder will retrieve the Bing Search API Key

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.20.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.20.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.20.0-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Bing.WebSearch;
using System.ComponentModel;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.20.0 Microsoft.SemanticKernel.Plugins.Core, 1.20.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.20.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Using Azure OpenAI Service


### Step 2 - Create a custom WebSearch Results Plugin

Execute the next two cells to:
* Create a custom WebSearchResultsPlugin
* The plugin includes a native C# function called GetWebSearchResults
* GetWebSearchResults uses the Bing Search SDK to perform an advanced web search
* Notice that the GetWebSearchResults method is decorated with the KernelFunction attribute. This lets Semantic Kernel know that this is a kernel function that can be imported  
* Both the GetWebSearchResults method and the nameOfBaseballPLayer parameter have descriptions  

In [3]:
// Simple class to hold the web search results
public class WebSearchResult
{
    public int Id { get; set; } = 0;
    public string Name { get; set; } = string.Empty;
    public string Snippet { get; set; } = string.Empty;
    public string Url { get; set; } = string.Empty;
}

// Custom Web Search Results Plugin
// Notice the code has much more functionality than the previous example
public class WebSearchResultsPlugin
{
    [KernelFunction]
    [Description("Retrieves the web search results with identified URL sources for a given baseball player.")]
    public async Task<string> GetWebSearchResults(
        [Description("Name of the baseball player to search.")]
        string nameOfBaseballPlayer)
    {
        // Config is just done to keep things simple for the example
        var configurationBuilder = new ConfigurationBuilder()
            .SetBasePath(Directory.GetCurrentDirectory())
            .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
            .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
        var config = configurationBuilder.Build();
        var bingSearchAPIKey = config["BingSearch:APIKey"];

        var webSearchResultsString = "Web search results:\r\n\r\n";
        var footNotes = string.Empty;
        var bingSearchId = 0;
        var webSearchResults = new List<WebSearchResult>();

        var bingSearchClient = new WebSearchClient(new ApiKeyServiceClientCredentials(bingSearchAPIKey));
        var bingWebData = await bingSearchClient.Web.SearchAsync(query: "baseball hall of fame " + nameOfBaseballPlayer, count: 8);

        if (bingWebData?.WebPages?.Value?.Count > 0)
        {
            // Itertate over the Bing Web Pages (Non-Cache Results)
            foreach (var bingWebPage in bingWebData.WebPages.Value)
            {
                bingSearchId++;

                webSearchResultsString += string.Format("[{0}]: \"{1}: {2}\"\r\nURL: {3}\r\n\r\n",
                    bingSearchId, bingWebPage.Name, bingWebPage.Snippet, bingWebPage.Url);

                footNotes += string.Format("[{0}]: {1}: {2}  \r\n",
                    bingSearchId, bingWebPage.Name, bingWebPage.Url);

                webSearchResults.Add(new WebSearchResult
                {
                    Id = bingSearchId,
                    Name = bingWebPage.Name,
                    Snippet = bingWebPage.Snippet,
                    Url = bingWebPage.Url
                });
            }
        }

        return webSearchResultsString;
    }
}
semanticKernel.ImportPluginFromType<WebSearchResultsPlugin>();

### Step 3 - Recommend a Decision using the Internet Knowledge Graph
> "Knowledge is power, but only if you use it to make decisions."
>
> -- <cite>Unknown</cite> 

The custom plugin above performs an advanced Bing Search query, maintains the sources (sites) that were used for search and keeps the index of each site used. This is important as it not only provides knowledge grounding information but it also provides a framework to create citations as the LLM reasons over the decision recommendation. This allows AI architects to build "AI Decision explainability" into their AI orchestration pipelines.

Execute the cell below to understand the LLMs reasoning process, it's final decision recommendation on Mike Trout's hall of fame induction and finally the sources (citations) that were used to arrive at that conclusion.

In [4]:
// Configure the OpenAI prompt execution settings with Auto Invoking of Kernel Functions
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.4, // balance between creativity and relevance
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Provide sources of information and explain your reasoning.
Ensure to cite all the sources using [number] notation of each URL after the reference in order.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let's invoke the prompt and stream the results
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Determining whether Mike Trout should be inducted into the Hall of Fame involves evaluating several key factors, including his career statistics, accolades, and recent performance trends, particularly in light of injuries. Here’s a detailed analysis based on the information gathered:

### Career Achievements and Statistics
1. **Accolades**: Trout is a three-time MVP, a 14-time All-Star, and has won nine Silver Slugger awards. These honors place him among the elite players in baseball history, showcasing his consistent excellence over the years [1][8].
   
2. **Statistical Milestones**: As of now, Trout has impressive career statistics, including a high on-base percentage and slugging percentage, which are critical metrics for evaluating a player's offensive contribution. His ability to maintain a high level of performance over many seasons is a strong argument for his Hall of Fame candidacy [1][8].

### Impact of Injuries
1. **Injury Concerns**: Trout has faced significant injury issue